In [1]:
import sys
sys.path.insert(0,'../../backend')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import gym
import stable_baselines
from stable_baselines import A2C, ACER, ACKTR, DQN, DDPG, SAC, PPO1, PPO2, TD3, TRPO
from stable_baselines.common.env_checker import check_env
from stable_baselines.common.policies import MlpPolicy
from stable_baselines.gail import ExpertDataset, generate_expert_traj

import webotsgym as wg

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [2]:
from webotsgym.utils import exponential_decay
from webotsgym.env.reward import step_pen_exp

class MyEval(wg.WbtReward):
    def __init__(self, env, config):
        super(MyEval, self).__init__(env, config)

    def calc_reward(self):
        target_distance = self.env.get_target_distance(False)
        if target_distance < 0.08:
            return 5000 + 5000 * (1 - abs(self.env.state.speed))
        else:
            reward = 0
            reward += -2
        
            #if self.env.state._action_denied:
               # reward += -3
        
            if self.env.state.touching:
                reward += -100
        return reward

    def check_done(self):
        if self.env.total_reward < -5000:
            print("reward boundary")
            return True
        if self.env.get_target_distance(False) < 0.08:
            print("target reached")
            return True
        return False

In [3]:
config = wg.WbtConfig()
config.world_size = 3
config.num_obstacles = 0
config.sim_mode = wg.config.SimSpeedMode.FAST
config.sim_step_every_x = 5
config.relative_action = True
config.direction_type = wg.config.DirectionType.STEERING

SimSpeedMode.FAST


In [4]:
model_name = "3x3_reach_target_v3"
env = wg.WbtGym(train=True, 
                evaluate_class=MyEval,
#                 action_class = wg.WbtActDiscrete(config, dspeed=0.05, ddir=0.05),
                config=config)

/home/jonas/anaconda3/lib/python3.6/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
../../backend/webotsgym/env/webotenv.py:112: UserWarning: Relative property of action class is overwritten by config.relative_action.
  warnings.warn("Relative property of action class is overwritten by config.relative_action.")


Accepting on Port:  10201


In [5]:
model = PPO1("MlpPolicy", env, tensorboard_log="./{}".format(model_name))





Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Please use `layer.__call__` method instead.









Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [6]:
model.learn(total_timesteps=200000)
model.save("./{}".format(model_name))

AttributeError: 'WbtState' object has no attribute '_action_denied'